# <img style="float: left; padding-right: 0.7cm; width: 3.0cm" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/179px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"> IMT 2210 - 1 Algebra lineal para Ciencias de Datos.
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2022-2**<br>
**Profesor** Cristobal rojas <br>

----